<a href="https://colab.research.google.com/github/ElenaNtm/IPTO-Data-projections/blob/main/ipto_data_before_may_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Libraries

In [ ]:
import pandas as pd
#import requests
from datetime import datetime, timedelta
#from datetime import datetime, timedelta
from urllib.error import HTTPError

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


#Preliminaries

In [ ]:
#For the urls that can not be read we will keep the date
not_found = []
# Initialize empty DataFrames and do all the jobs at once, so as to download only once the file
df_hydro = pd.DataFrame()
df_lignite = pd.DataFrame()
df_net_load = pd.DataFrame()
df_gas = pd.DataFrame()
df_res = pd.DataFrame()
df_inter = pd.DataFrame()

In [ ]:
base_url = "https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/"

#2018-2019

In [ ]:
months = [1,2,3,4,5,6,7,8,9,10,11,12]
days_in_month = {
    1: 31,  # January
    2: 28,  # February (non-leap year)
    3: 31,  # March
    4: 30,  # April
    5: 31,  # May
    6: 30,  # June
    7: 31,  # July
    8: 31,  # August
    9: 30,  # September
    10: 31,  # October
    11: 30,  # November
    12: 31,  # December
}

In [ ]:
for i in months:
   start_date = datetime(2018, i, 1)
   end_date = datetime(2018, i, days_in_month[i])
   date_format = "%Y%m%d"
   current_date = start_date

   while current_date <= end_date:
       date_str = current_date.strftime(date_format)
       url = f"{base_url}{date_str}_SystemRealizationSCADA_01.xls"

       try:
           # Read the XLSX file into a pandas DataFrame
           df1 = pd.read_excel(url, sheet_name="System_Production", converters={'(ΣΤΟΙΧΕΙΑ SCADA ΜΗ-ΠΙΣΤΟΠΟΙΗΜΕΝΑ)':str})
           #Hydro
           substring = 'ΣΥΝΟΛΟ ΥΔΡΟΗΛΕΚΤΡΙΚΩΝ'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'TOTAL HYDRO'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total Hydro'], index=[date_str])
           df_hydro = pd.concat([df_hydro, row])
           df_hydro = df_hydro.rename(index={wanted_index[len(wanted_index)-1]: date_str})

           #Lignite
           substring = 'ΣΥΝΟΛΟ ΛΙΓΝΙΤΙΚΩΝ'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'TOTAL LIGNITE'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total Lignite in MWh'], index=[date_str])

           df_lignite = pd.concat([df_lignite, row])
           df_lignite = df_lignite.rename(index={wanted_index[len(wanted_index)-1]: date_str})

           #Gas
           substring = 'ΣΥΝΟΛΟ Φ. ΑΕΡΙΟΥ'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'TOTAL GAS'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total Gas'], index=[date_str])

           df_gas = pd.concat([df_gas, row])
           df_gas = df_gas.rename(index={wanted_index[len(wanted_index)-1]: date_str})


           #RES
           substring = 'TOTAL RES'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'ΣΥΝΟΛΟ ΑΙΟΛΙΚΩΝ'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total RES'], index=[date_str])

           df_res = pd.concat([df_res, row])
           df_res = df_res.rename(index={wanted_index[len(wanted_index)-1]: date_str})

           #Net Load
           substring = 'NET LOAD'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'ΚΑΘΑΡΟ ΦΟΡΤΙΟ'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total Net Load'], index=[date_str])

           df_net_load = pd.concat([df_net_load, row])
           df_net_load = df_net_load.rename(index={wanted_index[len(wanted_index)-1]: date_str})

           #Interconnections
           substring = 'EXPORTS-IMPORTS'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'ΚΑΘΑΡΟ ΔΙΑΣΥΝΔΕΣΕΩΝ'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Exports - Imports'], index=[date_str])

           df_inter = pd.concat([df_inter, row])
           df_inter = df_inter.rename(index={wanted_index[len(wanted_index)-1]: date_str})
       except HTTPError as e:
           print(f"Error accessing URL: {url}")
           print(e)
           not_found.append(current_date)

       # Increment to the next date
       current_date += timedelta(days=1)


Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/20180305_SystemRealizationSCADA_01.xls
HTTP Error 404: Not Found
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/20180325_SystemRealizationSCADA_01.xls
HTTP Error 404: Not Found
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/20180424_SystemRealizationSCADA_01.xls
HTTP Error 404: Not Found
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/20180716_SystemRealizationSCADA_01.xls
HTTP Error 404: Not Found
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/20180723_SystemRealizationSCADA_01.xls
HTTP Error 404: Not Found
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/20180724_SystemRealizationSCADA_01.xls
HTTP Error 404: Not Found
Error accessing URL: https://www.admie.g

#2019

In [ ]:
months = [1,2,3,4,5,6,7,8,9,10,11,12]

for i in months:
   start_date = datetime(2019, i, 1)
   end_date = datetime(2019, i, days_in_month[i])
   date_format = "%Y%m%d"
   current_date = start_date

   while current_date <= end_date:
       date_str = current_date.strftime(date_format)
       url = f"{base_url}{date_str}_SystemRealizationSCADA_01.xls"

       try:
           # Read the XLSX file into a pandas DataFrame
           df1 = pd.read_excel(url, sheet_name="System_Production", converters={'(ΣΤΟΙΧΕΙΑ SCADA ΜΗ-ΠΙΣΤΟΠΟΙΗΜΕΝΑ)':str})
           #Hydro
           substring = 'ΣΥΝΟΛΟ ΥΔΡΟΗΛΕΚΤΡΙΚΩΝ'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'TOTAL HYDRO'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total Hydro'], index=[date_str])
           df_hydro = pd.concat([df_hydro, row])
           df_hydro = df_hydro.rename(index={wanted_index[len(wanted_index)-1]: date_str})

           #Lignite
           substring = 'ΣΥΝΟΛΟ ΛΙΓΝΙΤΙΚΩΝ'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'TOTAL LIGNITE'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total Lignite in MWh'], index=[date_str])

           df_lignite = pd.concat([df_lignite, row])
           df_lignite = df_lignite.rename(index={wanted_index[len(wanted_index)-1]: date_str})

           #Gas
           substring = 'ΣΥΝΟΛΟ Φ. ΑΕΡΙΟΥ'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'TOTAL GAS'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total Gas'], index=[date_str])

           df_gas = pd.concat([df_gas, row])
           df_gas = df_gas.rename(index={wanted_index[len(wanted_index)-1]: date_str})


           #RES
           substring = 'TOTAL RES'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'ΣΥΝΟΛΟ ΑΙΟΛΙΚΩΝ'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total RES'], index=[date_str])

           df_res = pd.concat([df_res, row])
           df_res = df_res.rename(index={wanted_index[len(wanted_index)-1]: date_str})

           #Net Load
           substring = 'NET LOAD'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'ΚΑΘΑΡΟ ΦΟΡΤΙΟ'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total Net Load'], index=[date_str])

           df_net_load = pd.concat([df_net_load, row])
           df_net_load = df_net_load.rename(index={wanted_index[len(wanted_index)-1]: date_str})
           #Interconnections
           substring = 'EXPORTS-IMPORTS'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'ΚΑΘΑΡΟ ΔΙΑΣΥΝΔΕΣΕΩΝ'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Exports - Imports'], index=[date_str])

           df_inter = pd.concat([df_inter, row])
           df_inter = df_inter.rename(index={wanted_index[len(wanted_index)-1]: date_str})
       except HTTPError as e:
           print(f"Error accessing URL: {url}")
           print(e)
           not_found.append(current_date)

       # Increment to the next date
       current_date += timedelta(days=1)


Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/20190116_SystemRealizationSCADA_01.xls
HTTP Error 404: Not Found
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/20190119_SystemRealizationSCADA_01.xls
HTTP Error 404: Not Found
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/20190120_SystemRealizationSCADA_01.xls
HTTP Error 404: Not Found
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/20190121_SystemRealizationSCADA_01.xls
HTTP Error 404: Not Found
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/20190417_SystemRealizationSCADA_01.xls
HTTP Error 404: Not Found
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/20191029_SystemRealizationSCADA_01.xls
HTTP Error 404: Not Found


#2020

In [ ]:
"""
2020 μας ενδιαφέρει μόνο μέχρι τον ιούλιο
Σπάει σε 2 καθώς ότι είναι από τον Μάιο και έπειτα έχουν άλλο url
"""
months=[1,2,3]
days_in_month = {
    1: 31,  # January
    2: 29,  # February (leap year)
    3: 31,  # March
    4: 30,  # April
    5: 31,  # May
    6: 30,  # June
    7: 31,  # July
    8: 31,  # August
    9: 30,  # September
    10: 31,  # October
    11: 30,  # November
    12: 31,  # December
}

In [ ]:
for i in months:
   start_date = datetime(2020, i, 1)
   end_date = datetime(2020, i, days_in_month[i] )
   date_format = "%Y%m%d"
   current_date = start_date

   while current_date <= end_date:
       date_str = current_date.strftime(date_format)
       url = f"{base_url}{date_str}_SystemRealizationSCADA_01.xls"

       try:
           # Read the XLSX file into a pandas DataFrame
           df1 = pd.read_excel(url, sheet_name="System_Production", converters={'(ΣΤΟΙΧΕΙΑ SCADA ΜΗ-ΠΙΣΤΟΠΟΙΗΜΕΝΑ)':str})
           #Hydro
           substring = 'ΣΥΝΟΛΟ ΥΔΡΟΗΛΕΚΤΡΙΚΩΝ'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'TOTAL HYDRO'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total Hydro'], index=[date_str])
           df_hydro = pd.concat([df_hydro, row])
           df_hydro = df_hydro.rename(index={wanted_index[len(wanted_index)-1]: date_str})

           #Lignite
           substring = 'ΣΥΝΟΛΟ ΛΙΓΝΙΤΙΚΩΝ'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'TOTAL LIGNITE'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total Lignite in MWh'], index=[date_str])

           df_lignite = pd.concat([df_lignite, row])
           df_lignite = df_lignite.rename(index={wanted_index[len(wanted_index)-1]: date_str})

           #Gas
           substring = 'ΣΥΝΟΛΟ Φ. ΑΕΡΙΟΥ'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'TOTAL GAS'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total Gas'], index=[date_str])

           df_gas = pd.concat([df_gas, row])
           df_gas = df_gas.rename(index={wanted_index[len(wanted_index)-1]: date_str})


           #RES
           substring = 'TOTAL RES'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'ΣΥΝΟΛΟ ΑΙΟΛΙΚΩΝ'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total RES'], index=[date_str])

           df_res = pd.concat([df_res, row])
           df_res = df_res.rename(index={wanted_index[len(wanted_index)-1]: date_str})

           #Net Load
           substring = 'NET LOAD'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'ΚΑΘΑΡΟ ΦΟΡΤΙΟ'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total Net Load'], index=[date_str])

           df_net_load = pd.concat([df_net_load, row])
           df_net_load = df_net_load.rename(index={wanted_index[len(wanted_index)-1]: date_str})
           #Interconnections
           substring = 'EXPORTS-IMPORTS'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'ΚΑΘΑΡΟ ΔΙΑΣΥΝΔΕΣΕΩΝ'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Exports - Imports'], index=[date_str])

           df_inter = pd.concat([df_inter, row])
           df_inter = df_inter.rename(index={wanted_index[len(wanted_index)-1]: date_str})
       except HTTPError as e:
           print(f"Error accessing URL: {url}")
           print(e)
           not_found.append(current_date)

       # Increment to the next date
       current_date += timedelta(days=1)

Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/20200201_SystemRealizationSCADA_01.xls
HTTP Error 404: Not Found
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/20200327_SystemRealizationSCADA_01.xls
HTTP Error 404: Not Found
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/20200328_SystemRealizationSCADA_01.xls
HTTP Error 404: Not Found
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/20200329_SystemRealizationSCADA_01.xls
HTTP Error 404: Not Found
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/20200330_SystemRealizationSCADA_01.xls
HTTP Error 404: Not Found
Error accessing URL: https://www.admie.gr/sites/default/files/attached-files/type-file/2020/03/20200331_SystemRealizationSCADA_01.xls
HTTP Error 404: Not Found


The url format changes here

In [ ]:
base_url = 'https://www.admie.gr/sites/default/files/attached-files/type-file/2020/04/'

In [ ]:
start_date = datetime(2020, 3, 27)
end_date = datetime(2020, 3, 31)
date_format = "%Y%m%d"
current_date = start_date

In [ ]:
while current_date <= end_date:
    date_str = current_date.strftime(date_format)
    url = f"{base_url}{date_str}_SystemRealizationSCADA_01.xls"

    try:
        # Read the XLSX file into a pandas DataFrame
        df1 = pd.read_excel(url, sheet_name="System_Production", converters={'(ΣΤΟΙΧΕΙΑ SCADA ΜΗ-ΠΙΣΤΟΠΟΙΗΜΕΝΑ)':str})
        #Hydro
        substring = 'ΣΥΝΟΛΟ ΥΔΡΟΗΛΕΚΤΡΙΚΩΝ'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'TOTAL HYDRO'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Hydro'], index=[date_str])
        df_hydro = pd.concat([df_hydro, row])
        df_hydro = df_hydro.rename(index={wanted_index[len(wanted_index)-1]: date_str})

        #Lignite
        substring = 'ΣΥΝΟΛΟ ΛΙΓΝΙΤΙΚΩΝ'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'TOTAL LIGNITE'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Lignite in MWh'], index=[date_str])

        df_lignite = pd.concat([df_lignite, row])
        df_lignite = df_lignite.rename(index={wanted_index[len(wanted_index)-1]: date_str})

        #Gas
        substring = 'ΣΥΝΟΛΟ Φ. ΑΕΡΙΟΥ'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'TOTAL GAS'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Gas'], index=[date_str])

        df_gas = pd.concat([df_gas, row])
        df_gas = df_gas.rename(index={wanted_index[len(wanted_index)-1]: date_str})


        #RES
        substring = 'TOTAL RES'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'ΣΥΝΟΛΟ ΑΙΟΛΙΚΩΝ'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total RES'], index=[date_str])

        df_res = pd.concat([df_res, row])
        df_res = df_res.rename(index={wanted_index[len(wanted_index)-1]: date_str})

        #Net Load
        substring = 'NET LOAD'
        filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
        wanted_index = filtered.index.tolist()

        if not wanted_index:
            substring = 'ΚΑΘΑΡΟ ΦΟΡΤΙΟ'
            filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
            wanted_index = filtered.index.tolist()

        value = df1.iloc[wanted_index[len(wanted_index)-1],26]
        row = pd.DataFrame([[value]], columns=['Total Net Load'], index=[date_str])

        df_net_load = pd.concat([df_net_load, row])
        df_net_load = df_net_load.rename(index={wanted_index[len(wanted_index)-1]: date_str})
    except HTTPError as e:
        print(f"Error accessing URL: {url}")
        print(e)
        not_found.append(current_date)

    # Increment to the next date
    current_date += timedelta(days=1)
month = [4,5,6,7]
base_url = "https://www.admie.gr/sites/default/files/attached-files/type-file/2020/07/"
for i in month:
   start_date = datetime(2020, 7, 29)
   end_date = datetime(2020, 7, 30)
   date_format = "%Y%m%d"
   current_date = start_date

   while current_date <= end_date:
       date_str = current_date.strftime(date_format)
       url = f"{base_url}{date_str}_SystemRealizationSCADA_01.xls"

       try:
           # Read the XLSX file into a pandas DataFrame
           df1 = pd.read_excel(url, sheet_name="System_Production", converters={'(ΣΤΟΙΧΕΙΑ SCADA ΜΗ-ΠΙΣΤΟΠΟΙΗΜΕΝΑ)':str})
           #Hydro
           substring = 'ΣΥΝΟΛΟ ΥΔΡΟΗΛΕΚΤΡΙΚΩΝ'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'TOTAL HYDRO'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total Hydro'], index=[date_str])
           df_hydro = pd.concat([df_hydro, row])
           df_hydro = df_hydro.rename(index={wanted_index[len(wanted_index)-1]: date_str})

           #Lignite
           substring = 'ΣΥΝΟΛΟ ΛΙΓΝΙΤΙΚΩΝ'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'TOTAL LIGNITE'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total Lignite in MWh'], index=[date_str])

           df_lignite = pd.concat([df_lignite, row])
           df_lignite = df_lignite.rename(index={wanted_index[len(wanted_index)-1]: date_str})

           #Gas
           substring = 'ΣΥΝΟΛΟ Φ. ΑΕΡΙΟΥ'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'TOTAL GAS'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total Gas'], index=[date_str])

           df_gas = pd.concat([df_gas, row])
           df_gas = df_gas.rename(index={wanted_index[len(wanted_index)-1]: date_str})


           #RES
           substring = 'TOTAL RES'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'ΣΥΝΟΛΟ ΑΙΟΛΙΚΩΝ'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total RES'], index=[date_str])

           df_res = pd.concat([df_res, row])
           df_res = df_res.rename(index={wanted_index[len(wanted_index)-1]: date_str})

           #Net Load
           substring = 'NET LOAD'
           filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
           wanted_index = filtered.index.tolist()

           if not wanted_index:
               substring = 'ΚΑΘΑΡΟ ΦΟΡΤΙΟ'
               filtered = df1[df1.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
               wanted_index = filtered.index.tolist()

           value = df1.iloc[wanted_index[len(wanted_index)-1],26]
           row = pd.DataFrame([[value]], columns=['Total Net Load'], index=[date_str])

           df_net_load = pd.concat([df_net_load, row])
           df_net_load = df_net_load.rename(index={wanted_index[len(wanted_index)-1]: date_str})
       except HTTPError as e:
           print(f"Error accessing URL: {url}")
           print(e)
           not_found.append(current_date)

       # Increment to the next date
       current_date += timedelta(days=1)

In [ ]:
for i in not_found:
    print(i)

2018-03-05 00:00:00
2018-03-25 00:00:00
2018-04-24 00:00:00
2018-07-16 00:00:00
2018-07-23 00:00:00
2018-07-24 00:00:00
2018-07-25 00:00:00
2018-09-13 00:00:00
2018-11-20 00:00:00
2018-12-18 00:00:00
2018-12-23 00:00:00
2019-01-16 00:00:00
2019-01-19 00:00:00
2019-01-20 00:00:00
2019-01-21 00:00:00
2019-04-17 00:00:00
2019-10-29 00:00:00
2020-02-01 00:00:00
2020-03-27 00:00:00
2020-03-28 00:00:00
2020-03-29 00:00:00
2020-03-30 00:00:00
2020-03-31 00:00:00


#Drop duplicates

In [ ]:
#RES
df_res['index'] = df_res.index
df_res = df_res.drop_duplicates(subset='index')
df_res.drop(['index'], axis = 1, inplace = True)

#HYDRO
df_hydro['index'] = df_hydro.index
df_hydro = df_hydro.drop_duplicates(subset='index')
df_hydro.drop(['index'], axis = 1, inplace = True)

#LIGNITE
df_lignite['index'] = df_lignite.index
df_lignite = df_lignite.drop_duplicates(subset='index')
df_lignite.drop(['index'], axis = 1, inplace = True)

#NET LOAD
df_net_load['index'] = df_net_load.index
df_net_load = df_net_load.drop_duplicates(subset='index')
df_net_load.drop(['index'], axis = 1, inplace = True)

#GAS
df_gas['index'] = df_gas.index
df_gas = df_gas.drop_duplicates(subset='index')
df_gas.drop(['index'], axis = 1, inplace = True)

#Interconnections
df_inter['index'] = df_inter.index
df_inter = df_inter.drop_duplicates(subset='index')
df_inter.drop(['index'], axis = 1, inplace = True)

not_found = list(set(not_found))

<ipython-input-19-af4abcfe29a1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res.drop(['index'], axis = 1, inplace = True)
<ipython-input-19-af4abcfe29a1>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hydro.drop(['index'], axis = 1, inplace = True)
<ipython-input-19-af4abcfe29a1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lignite.drop(['index'], axis = 1, inplace = True)
<ipython-input-19-af4abcfe29a1>:19: Settin

#Indexes datetime format

In [ ]:
# Convert the index to datetime
df_res.index = pd.to_datetime(df_res.index, format='%Y/%m/%d')
df_lignite.index = pd.to_datetime(df_lignite.index, format='%Y/%m/%d')
df_net_load.index = pd.to_datetime(df_net_load.index, format='%Y/%m/%d')
df_hydro.index = pd.to_datetime(df_hydro.index, format='%Y/%m/%d')
df_gas.index = pd.to_datetime(df_gas.index, format='%Y/%m/%d')
df_inter.index = pd.to_datetime(df_inter.index, format='%Y/%m/%d')

#Column names

In [ ]:
df_res.index.name = 'Date'
df_net_load.index.name = 'Date'
df_lignite.index.name = 'Date'
df_hydro.index.name = 'Date'
df_gas.index.name = 'Date'
df_inter.index.name = 'Date'

#Missing Data

In [ ]:
#Join all to one dataframe to see for missing data
df = pd.DataFrame()
df = df_gas.join(df_res,on = 'Date', how = 'outer')
df = df.join(df_hydro,on = 'Date', how = 'outer')
df = df.join(df_lignite,on = 'Date', how = 'outer')
df = df.join(df_net_load, on = 'Date',how = 'outer')
df = df.join(df_inter, on = 'Date', how = 'outer')
df.reset_index(drop=False, inplace=True)
df.head()
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(10)
#df.tail(20)
#df_net_load.tail(20)

,Total,Percent
Exports - Imports,7,0.008696
Date,0,0.000000
Total Gas,0,0.000000
Total RES,0,0.000000
Total Hydro,0,0.000000
Total Lignite in MWh,0,0.000000
Total Net Load,0,0.000000


In [ ]:
df.head()

,Date,Total Gas,Total RES,Total Hydro,Total Lignite in MWh,Total Net Load,Exports - Imports
0,2018-01-01,28292,11706,6063,42864,116396,-29632.0
1,2018-01-02,34403,19014,6701,42972,126445,-24058.0
2,2018-01-03,39057,12966,6839,48943,136125,-28320.0
3,2018-01-04,45362,21505,9021,52584,149877,-23022.0
4,2018-01-05,48109,7269,8324,52834,144209,-28956.0


#Save to drive

In [ ]:
df.to_csv('data_from2018.csv')
!cp data_from2018.csv "drive/My Drive/"